In [8]:
import pickle
from pathlib import Path
import yaml

import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run
from neuralhydrology.utils.config import Config

In [2]:
# by default we assume that you have at least one CUDA-capable NVIDIA GPU or MacOS with Metal support
if torch.cuda.is_available() or torch.backends.mps.is_available():
    start_run(config_file=Path("1_basin.yml"))

# fall back to CPU-only mode
else:
    start_run(config_file=Path("1_basin.yml"), gpu=-1)

2025-05-11 01:26:53,358: Logging to d:\github\neuralhydrology\neuralhydrology\own\LSTMAttention\runs\test_run_1105_012653\output.log initialized.
2025-05-11 01:26:53,358: ### Folder structure created at d:\github\neuralhydrology\neuralhydrology\own\LSTMAttention\runs\test_run_1105_012653
2025-05-11 01:26:53,358: ### Run configurations for test_run
2025-05-11 01:26:53,358: experiment_name: test_run
2025-05-11 01:26:53,368: train_basin_file: 1_basin.txt
2025-05-11 01:26:53,368: validation_basin_file: 1_basin.txt
2025-05-11 01:26:53,369: test_basin_file: 1_basin.txt
2025-05-11 01:26:53,369: train_start_date: 1999-10-01 00:00:00
2025-05-11 01:26:53,370: train_end_date: 2008-09-30 00:00:00
2025-05-11 01:26:53,371: validation_start_date: 1980-10-01 00:00:00
2025-05-11 01:26:53,372: validation_end_date: 1989-09-30 00:00:00
2025-05-11 01:26:53,372: test_start_date: 1989-10-01 00:00:00
2025-05-11 01:26:53,373: test_end_date: 1999-09-30 00:00:00
2025-05-11 01:26:53,373: device: cpu
2025-05-11 01

KeyboardInterrupt: 

In [10]:
import pickle
from pathlib import Path
import yaml # Need to import yaml to load/save dictionaries to/from file
import os # Import os for creating directory if needed

import matplotlib.pyplot as plt
import torch
# You might not need these imports if not directly using metrics or eval_run in this script
# from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run # Only need start_run
# from neuralhydrology.utils.config import Config # Only need Config if you were interacting with the object directly

# Define the values you want to test for hyperparameters
learning_rates_to_test = [{0: 1e-3}, {0: 5e-4}, {0: 1e-4}] # Using dict format matching config
dropouts_to_test = [0.0, 0.2, 0.4]

# Define the path to your base configuration file
base_config_path = Path("1_basin.yml")

# Define a directory to save temporary config files for each trial
temp_config_dir = Path("temp_configs")
temp_config_dir.mkdir(exist_ok=True) # Create the directory if it doesn't exist

# --- Load the base config into a dictionary ONCE outside the loop ---
# This mutable dictionary will be our template for each trial
with open(base_config_path, 'r') as f:
    base_config_dict = yaml.safe_load(f)
# -----------------------------------------------------------------


# --- Hyperparameter Tuning Loop ---
# Loop through each combination of hyperparameters
for lr_config in learning_rates_to_test:
    for dropout_rate in dropouts_to_test:

        # --- Create a MODIFIED dictionary for this trial ---
        # Start with a copy of the base config dictionary for the current trial
        trial_config_dict = base_config_dict.copy()

        # Modify the dictionary with the current hyperparameters for this trial
        trial_config_dict['learning_rate'] = lr_config
        trial_config_dict['output_dropout'] = dropout_rate

        # --- IMPORTANT: Set a UNIQUE experiment_name and run_dir for each trial ---
        # This ensures each trial's results are saved in a separate folder
        # Replace periods with 'p' to make directory names valid
        exp_name = f"attention_lr{str(lr_config[0]).replace('.', 'p')}_dp{str(dropout_rate).replace('.', 'p')}"
        trial_config_dict['experiment_name'] = exp_name
        # Ensure the run directory is based on the unique experiment name
        # You might need to adjust the base path for run_dir based on your setup
        trial_config_dict['run_dir'] = str(Path("./runs") / exp_name) # Example: ./runs/attention_lr0p001_dp0p0

        # --- Save the modified dictionary to a temporary YAML file for this trial ---
        # Create a unique filename for the temporary config
        temp_config_filename = f"config_{exp_name}.yml"
        temp_config_path = temp_config_dir / temp_config_filename

        with open(temp_config_path, 'w') as f:
            yaml.dump(trial_config_dict, f)
        # -----------------------------------------------------------

        # Print the config file being used for this trial (optional, for debugging)
        print(f"Starting trial for config file: {temp_config_path}")
        print(f"Learning Rate: {lr_config[0]}, Dropout: {dropout_rate}") # Print specific values

        # Determine device based on availability
        if torch.cuda.is_available() or torch.backends.mps.is_available():
            device = None # Use default device handling
        else:
            device = -1 # Use CPU

        # Start the NeuralHydrology run using the temporary config file path
        start_run(config_file=temp_config_path, gpu=device) # Pass the Path to the file

        print("-" * 30) # Separator between trials

        # Optional: Clean up the temporary config file after the run
        # import os
        # os.remove(temp_config_path)


# --- End of Hyperparameter Tuning Loop ---

Starting trial for config file: temp_configs\config_attention_lr0p001_dp0p0.yml
Learning Rate: 0.001, Dropout: 0.0
2025-05-11 01:56:15,460: Logging to runs\attention_lr0p001_dp0p0\attention_lr0p001_dp0p0_1105_015615\output.log initialized.
2025-05-11 01:56:15,460: ### Folder structure created at runs\attention_lr0p001_dp0p0\attention_lr0p001_dp0p0_1105_015615
2025-05-11 01:56:15,460: ### Run configurations for attention_lr0p001_dp0p0
2025-05-11 01:56:15,460: batch_size: 256
2025-05-11 01:56:15,460: clip_gradient_norm: 1
2025-05-11 01:56:15,460: clip_targets_to_zero: ['QObs(mm/d)']
2025-05-11 01:56:15,460: data_dir: ..\..\data\CAMELS_US
2025-05-11 01:56:15,460: dataset: camels_us
2025-05-11 01:56:15,471: device: cpu
2025-05-11 01:56:15,471: dynamic_inputs: ['PRCP(mm/day)_nldas', 'PRCP(mm/day)_maurer', 'prcp(mm/day)_daymet', 'srad(W/m2)_daymet', 'tmax(C)_daymet', 'tmin(C)_daymet', 'vp(Pa)_daymet']
2025-05-11 01:56:15,472: epochs: 50
2025-05-11 01:56:15,472: experiment_name: attention_lr0

KeyboardInterrupt: 